In [ ]:
from google.colab import drive
drive.mount('/content/drive')

address = "Colab Notebooks"
import sys
sys.path.append('/content/drive/My Drive/{}'.format(address))

%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

#Base model (using Fasttext Embeddings)

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199779 sha256=59e9080aade8ca6b0fb07f27adc78435a932c504089669871ba4fbef5b2f3e12
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
import nltk
import string
from nltk.corpus import stopwords
import re
import spacy
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Save and train the fasttext model

### Make dataset ready and save it

In [ ]:
movies_df = pd.read_csv('./Movies.csv')
movies_df.iloc[100:105]

,Unnamed: 0,Genre,Origin/Ethnicity,Plot,Release Year,Title
100,100,drama,American,"Grace Roberts (played by Lea Leland), marries ...",1914,Should a Woman Divorce?
101,101,western,American,James Wynnegate (Dustin Farnum) and his cousin...,1914,The Squaw Man
102,102,unknown,American,"Sad, lonely and unhappy, an old man sits in a ...",1914,Sweet and Low
103,103,comedy,American,The Masher played by Charlie Chaplin fights fo...,1914,Those Love Pangs
104,104,comedy,American,Charles Chaplin portrays a womanizing city man...,1914,Tillie's Punctured Romance


In [ ]:
stopwords_collection = set(stopwords.words('english'))

def preprocess_text(text):
    words = word_tokenize(text.lower())
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    words = [word for word in words if word not in stopwords_collection]
    doc = nlp(' '.join(words))
    lemmatized_words = [token.lemma_ for token in doc]
    return ' '.join([word for word in lemmatized_words if len(word) >= 2])

from tqdm import tqdm

all_text_cleaned = [preprocess_text(plot + " " + title) for plot, title in tqdm(zip(movies_df['Plot'], movies_df['Title']), total=len(movies_df))]

  0%|          | 114/36323 [00:08<50:57, 11.84it/s]  

KeyboardInterrupt: 

In [ ]:
with open("all_text_cleaned.txt", "w") as file:
    file.write("===\n".join(all_text_cleaned))

NameError: name 'all_text_cleaned' is not defined

### Train a fasttext model

In [ ]:
all_text_cleaned = []
with open("all_text_cleaned.txt", "r") as file:
    content = file.read()
    all_text_cleaned = content.split("===")

all_text_cleaned = [item.strip() for item in all_text_cleaned if item.strip()]

In [ ]:
print(all_text_cleaned[50])

physician death orphan two adolescent daughter old brother able convert doctor small estate cash late day bank close store money father household safe slatternly housekeeper aware money enlist criminal acquaintance help crack safe lock daughter adjacent room drunken housekeeper menace brandish gun hole wall resourceful girl use telephone call brother return town get message organize rescue party    unseen enemy


In [ ]:
class FastText:
    def __init__(self, method='skipgram'):
        self.method = method
        self.model = None

    def train(self, texts):
        with open('training_data.txt', 'w', encoding='utf-8') as file:
            for t in texts:
                file.write(t + '\n')
        self.model = fasttext.train_unsupervised('training_data.txt', model=self.method, dim=300)

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)

fasttext_model = FastText()

In [ ]:
fasttext_model.train(all_text_cleaned)
fasttext_model.save_FastText_model()

NameError: name 'fasttext' is not defined

In [ ]:
fasttext_model.load_FastText_model()

word_to_query = 'laptop'
num_neighbors = 10
nearest_neighbors = fasttext_model.model.get_nearest_neighbors(word_to_query, k=num_neighbors)

print(f"Nearest neighbors of '{word_to_query}':")
for neighbor, similarity in nearest_neighbors:
    print(f"{neighbor}: {similarity}")

Nearest neighbors of 'laptop':
0.5844805836677551: database
0.5773231387138367: desktop
0.5691365003585815: cellphone
0.5649459362030029: upload
0.5645779371261597: download
0.5615304708480835: monogrammed
0.5611780881881714: uploaded
0.5550886988639832: usb
0.5547577738761902: audiotape
0.5524214506149292: vcr


## Use TF-IDF weighting to find sentense embedding

In [ ]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def fit_vectorizer(self, data):
        self.vectorizer.fit(data)

    def transform(self, data):
        tfidf_matrix = self.vectorizer.transform(data)
        return tfidf_matrix

In [ ]:
tfidf = TF_IDF()

tfidf.fit_vectorizer(all_text_cleaned)
tfidf_matrix = tfidf.transform(all_text_cleaned)

feature_names = tfidf.vectorizer.get_feature_names_out()

document_index = 0
tfidf_representation = tfidf_matrix[document_index]

tfidf_values = tfidf_representation.toarray()[0]
top_n_indices = tfidf_values.argsort()[-10:][::-1]
print(f"\nTop 10 Words with Highest TF-IDF Values for Document at index: {document_index}:")
for index in top_n_indices:
    print(f"{feature_names[index]}: {tfidf_values[index]}     {index}")


Top 10 Words with Highest TF-IDF Values for Document at index: 0:
bartender: 0.29308512456178026     13642
beer: 0.28166509288249064     14580
saloon: 0.27548777678209346     116420
irish: 0.2753074211837885     64072
nation: 0.2624965967846531     92044
smasher: 0.22456260882197238     125172
seltzer: 0.22099701477824932     119914
stereotypically: 0.20852971515457158     128604
fixture: 0.19558547953736938     46086
carrie: 0.16523385229838966     22851


In [ ]:
def get_sentence_embedding(model, sentence, tf_idf_vectorizer):
    w = tf_idf_vectorizer.transform([sentence])
    embedding = np.zeros(300)
    for word in list(set(sentence.split())):
        try:
            embedding += model.get_word_vector(word) * w[0, tfidf.vectorizer.vocabulary_[word]]
        except:
            pass
    return embedding

In [ ]:
def find_two_queries_similarity(query1, query2):
  query_embedding1 = get_sentence_embedding(fasttext_model.model, query1.lower(), tfidf.vectorizer)
  query_embedding2 = get_sentence_embedding(fasttext_model.model, query2.lower(), tfidf.vectorizer)
  return cosine_similarity([query_embedding1], [query_embedding2])[0][0]

find_two_queries_similarity("young boy is playing", "happy kid is playing")

0.8416289399823016

In [ ]:
document_embeddings = [get_sentence_embedding(fasttext_model.model, doc, tfidf.vectorizer) for doc in all_text_cleaned]

## Retrieve using these embeddings

In [ ]:
movies_df_with_embeddings = movies_df.copy()
movies_df_with_embeddings['emb'] = document_embeddings
movies_df_with_embeddings.head()

,Unnamed: 0,Genre,Origin/Ethnicity,Plot,Release Year,Title,emb
0,0,unknown,American,"A bartender is working at a saloon, serving dr...",1901,Kansas Saloon Smashers,"[0.2348172904457897, -0.4755801953142509, 0.58..."
1,1,unknown,American,"The moon, painted with a smiling face hangs ov...",1901,Love by the Light of the Moon,"[0.13217403783346526, -0.5993902126792818, 1.0..."
2,2,unknown,American,"The film, just over a minute long, is composed...",1901,The Martyred Presidents,"[0.4287504331150558, -0.3087563479784876, -0.1..."
3,3,unknown,American,Lasting just 61 seconds and consisting of two ...,1901,"Terrible Teddy, the Grizzly King","[0.7681060490140226, -0.641999500541715, 0.540..."
4,4,unknown,American,The earliest known adaptation of the classic f...,1902,Jack and the Beanstalk,"[0.7584390750853345, -0.46763472887687385, 0.9..."


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_movies_fasttext(query, k=10):
    query_embedding = get_sentence_embedding(fasttext_model.model, query.lower(), tfidf.vectorizer)
    embeddings_matrix = list(movies_df_with_embeddings['emb'])
    similarities = cosine_similarity([query_embedding], embeddings_matrix)[0]

    top_k_indices = similarities.argsort()[-k:][::-1]

    top_k_movies = movies_df_with_embeddings.iloc[top_k_indices][['Title', 'Plot']]

    return top_k_movies

In [ ]:
print(cosine_similarity([movies_df_with_embeddings.iloc[16389]['emb']], [movies_df_with_embeddings.iloc[12191]['emb']])[0][0])

0.9127352505115243


In [ ]:
def retrieve_multiple_queries(queries):
  all_results = []
  for query in queries:
    recommended_movies = retrieve_movies_fasttext(query, 2)
    json_results = recommended_movies.to_json(orient='records')
    all_results.append(json_results)
  return all_results

In [ ]:
import json

json_final_result = retrieve_multiple_queries(
["Find me a comedy that deals with college life like 'Revenge of the Nerds'.",
"List all the movies that explore themes of survival on an island similar to 'Swiss Family Robinson'.",
"Which movies, akin to 'Girl with a Pearl Earring', delve into the lives of famous painters?",
"I'm looking for movies based on classic literature like 'The Great Gatsby'.",
"What other films capture the essence of Christmas as 'The Muppet Christmas Carol' does?",
"Are there any films that center around a mystery at the circus in the same vein as 'Charlie Chan at the Circus'?",
"Provide a list of movies that showcase elaborate heists or capers akin to 'Tower Heist'.",
"What films offer narratives about rural America in the same way 'Lean on Pete' does?",
"Identify biopics that, similar to 'The Dolly Sisters', feature the lives of entertainment icons from the Vaudeville era.",
"Suggest movies that give a comedic take on espionage and secret agents as seen in 'Spy'.",
"Show me films where ancient artifacts bring history to life in a modern setting.",
"What are some action-packed movies that explore the theme of lone vigilantes seeking justice?",
"I'm interested in cinematic tales of forbidden love set against the backdrop of political upheaval.",
"Can you provide a list of horror sequels that delve into the aftermath of a curse from the first installment?",
"I'm looking for heartwarming family films where unexpected individuals come into each other’s lives and change them forever.",
"Identify movies depicting the struggles of space exploration and interstellar missions.",
"List all films that feature epic martial arts tournaments and the journey of the fighters.",
"Give me stories of cowboys taming the wild frontier in the old American West.",
"What movies offer a vivid portrayal of life during the Renaissance period?",
"I need films that center around a powerful artifact with mystical or historical significance.",
"Recommend romantic films where lovers deal with the challenges of distance or separation.",
"Find movies that involve high-stakes courtroom drama and legal battles.",
"Provide examples of comedy films based on the misadventures of individuals adapting to a new country.",
"I'm looking for spy thrillers that feature gadgetry and covert operations.",
"Show me horror films where the characters are trapped in a remote and haunted location.",
"Point me to movies showcasing the thrill of race car driving and the lives of drivers.",
"I want adventure films that revolve around treasure hunts and ancient civilizations.",
"List movies where the protagonist is a genius trying to solve a complex mystery.",
"Suggest musicals that capture the journey of aspiring performers from their humble beginnings to stardom.",
"Look for biographical films that illustrate the accomplishments and challenges of world-renowned scientists."]
)
with open('json_final_result_fasttext', 'w') as json_file:
    json.dump(json_final_result, json_file)

In [ ]:
query = "action movie that the main character is female"
recommended_movies = retrieve_movies_fasttext(query, 2)
json_results = recommended_movies.to_json(orient='records')
json_results

'[{"Title":"A Cock and Bull Story","Plot":"The film depicts Steve Coogan playing himself as an arrogant actor with low self-esteem and a complicated love life. Coogan is playing the eponymous role in an adaptation of The Life and Opinions of Tristram Shandy, Gentleman being filmed at a stately home. He constantly spars with actor Rob Brydon, who is playing Uncle Toby and believes his role to be of equal importance to Coogan\'s, calling himself the \\"co-lead\\".\\r\\nThe film incorporates several sequences from Tristram Shandy. Not all of these are part of the film-within-the-film. The latter are limited to the story of Tristram\'s conception, birth and christening; Uncle Toby\'s experiences at the Battle of Namur and Tristram\'s sudden and accidental circumcision at the age of three. Uncle Toby\'s wooing of Widow Wadman (Gillian Anderson) takes place in a sequence dreamed by Steve Coogan and after the cast and crew have viewed the \\"completed\\" film ending, with Walter Shandy fainti